In [204]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [260]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'Los Angeles Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [136]:
folder_path = "/Users/benjamincheng/Downloads/processed_2019_2024_1.csv"
df = pd.read_csv(folder_path, index_col=0)
df = df.reset_index(drop=True)
dates = df['date_next'].unique()

# Define time zones
dates_utc = []
est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
utc_timezone = pytz.timezone('UTC')

threshold_value = pd.to_datetime('2020-06-27T03:55:00Z')

full = []
for date in dates:
    date = date[:10]
    # Create a datetime object for the given time in EST
    est_time = datetime.strptime(f'{date} 12:00 PM', '%Y-%m-%d %I:%M %p')
    est_time = est_timezone.localize(est_time)

    # Convert EST time to UTC
    utc_time = est_time.astimezone(utc_timezone)
    if utc_time >= threshold_value:
        final_time = utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        dates_utc.append(final_time)

# df['Teams'] = df['Teams'].map(mapping)
# teams = df['Teams']

In [257]:
KEY = 'dc34ee156925bf51d4a4c33c87d440fc'
SPORT = 'basketball_nba'
REGION = 'us'
MARKET = 'h2h'
FORMAT = 'decimal'
full = []
DATES = dates_utc
for DATE in DATES:
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{SPORT}/odds',
        params={
            'api_key': KEY,
            'regions': REGION,
            'markets': MARKET,
            'oddsFormat': FORMAT,
            'date': DATE,
        }
    ).json()
    for i in range(len(response['data'])):
        # [Timestamp, good team, evil team, fanduel odds good team wins, draftkings odds, caesars odds]
        #best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
        bookmakers = response['data'][i]['bookmakers']
        for index, data in enumerate(bookmakers):
            best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
            if response['data'][i]['bookmakers'][index]['title'] in ["FanDuel", "BetMGM", "DraftKings", ""]:
                for outcome in range(2):
                    odds = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['price']
                    current_odds = best_odds[outcome][4]
                    best_odds[outcome][0] = response['timestamp']
                    best_odds[outcome][1] = response['data'][i]['id']
                    best_odds[outcome][2] = response['data'][i]['bookmakers'][index]['title']
                    best_odds[outcome][3] = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][4] = odds
                full.extend(best_odds)

KeyboardInterrupt: 

In [227]:
timestamp = []
Id = []
sportsbook = []
team = []
odds = []
for i in range(len(full)):
    timestamp.append(full[i][0])
    Id.append(full[i][1])
    sportsbook.append(full[i][2])
    team.append(full[i][3])
    odds.append(full[i][4])
df = pd.DataFrame({"Timestamp": timestamp,"Id": Id, "Sportsbook": sportsbook, "Team": team, "Odds": odds})

KeyError: 0

In [286]:
grouped = df.groupby('Id')
group_sizes = grouped.size()

# Filter groups with less than 6 rows
ids_with_less_than_six_rows = group_sizes[group_sizes < 6].index.tolist()

print("Ids with less than 6 rows:")

ids_with_less_than_six_rows

Ids with less than 6 rows:


['00ef92fb68581783d035845b500392db',
 '042deb765d791a748fb2d708b5ccf3d4',
 '090ae7250473daf4cfb7acab62082d8d',
 '0a25fa87e6a970c44bd5770e4b54440f',
 '0acb786cec1a76fe87811b4b42b4061b',
 '0c6e83c1fcec925dfffa08501fffd6d0',
 '0cdbb3003d85dc6dacc30ca76b41f408',
 '0cea16d9e557b0a7390bc71130c22bbf',
 '1126fa50abd910e80868523920c4516d',
 '1389614ee65aa896d30046aba2b74315',
 '13d77cdd1721022e6900156c45cc3ac9',
 '13e09e6616b22e86ad8696e987efa9d2',
 '13fcbf6a0b5901f30ac4e7f94d28ce16',
 '1c158d2adf414bae414f6c46424ff247',
 '1fbb851566301fb51b4ac3112df84b87',
 '2214fe5511c5e82505567370d36fa572',
 '291781e08309934322de7153e4d6a25a',
 '2945c19c3cd670bf8b58a35ca07f23f9',
 '2a278c578e758efdd56f411d3f815b86',
 '2de13ea858890273f9c69f38cabd47d7',
 '31651d6e78da993fe16332d19e9a2659',
 '34d36536e4c2d081b2ed84cc730b60f5',
 '354362afab3e1cee4a1c2379ab5c0301',
 '38a162613cf944fb5354798efea7754e',
 '3a3bc45074f2a39fe4adad6a1aa97787',
 '3b51ad6334a2f3f22baf60da7bb06d63',
 '3e9e20284be9cdc4bd524b893ed0a0ca',
 

In [287]:
miss = []
for Id in ids_with_less_than_six_rows:
    group = df[df['Id'] == Id]
    sportsbooks = ["FanDuel", "BetMGM", "DraftKings"]
    
    # check which sportsbook is missing
    present_sportsbooks = group['Sportsbook'].unique()
    missing_sportsbooks = [sb for sb in sportsbooks if sb not in present_sportsbooks][0]

    teams = group['Team'].unique()
    for team in teams:
        missing = [None, None, None, None, float('-inf')]
        # calculate average odds 
        missing[0] = group['Timestamp'].iloc[0]
        missing[1] = Id
        missing[2] = missing_sportsbooks
        missing[3] = team
        missing[4] = round(group[group['Team'] == team]['Odds'].mean(), 2)
        miss.append(missing)
missing_df = pd.DataFrame(miss, columns=['Timestamp', 'Id', 'Sportsbook', 'Team', 'Odds'])

In [288]:
concat = pd.concat([df, missing_df], ignore_index=True)
concat = concat.sort_values(by=['Timestamp', 'Id'])
concat = concat.reset_index(drop=True)
full = concat.merge(concat,
               left_on=['Timestamp','Id', 'Sportsbook'],
               right_on = ['Timestamp','Id', 'Sportsbook'])
full = full.drop(full[full['Team_x'] == full['Team_y']].index)
full = full.reset_index(drop=True)

In [289]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2021-01-20T16:55:00Z,06340b4723931e9471b3df5aa8b169a2,FanDuel,Houston Rockets,2.88,Phoenix Suns,1.44
1,2021-01-20T16:55:00Z,06340b4723931e9471b3df5aa8b169a2,FanDuel,Phoenix Suns,1.44,Houston Rockets,2.88
2,2021-01-20T16:55:00Z,06340b4723931e9471b3df5aa8b169a2,DraftKings,Houston Rockets,2.88,Phoenix Suns,1.43
3,2021-01-20T16:55:00Z,06340b4723931e9471b3df5aa8b169a2,DraftKings,Phoenix Suns,1.43,Houston Rockets,2.88
4,2021-01-20T16:55:00Z,06340b4723931e9471b3df5aa8b169a2,BetMGM,Houston Rockets,2.90,Phoenix Suns,1.40
...,...,...,...,...,...,...,...
23645,2024-03-20T15:55:38Z,def47031598222435edee8a26a792e3f,DraftKings,Milwaukee Bucks,4.90,Boston Celtics,1.19
23646,2024-03-20T15:55:38Z,def47031598222435edee8a26a792e3f,BetMGM,Boston Celtics,1.18,Milwaukee Bucks,5.00
23647,2024-03-20T15:55:38Z,def47031598222435edee8a26a792e3f,BetMGM,Milwaukee Bucks,5.00,Boston Celtics,1.18
23648,2024-03-20T15:55:38Z,def47031598222435edee8a26a792e3f,FanDuel,Boston Celtics,1.17,Milwaukee Bucks,5.30


In [290]:
full['Timestamp'] = pd.to_datetime(full['Timestamp'])

utc_timezone = pytz.utc
est_timezone = pytz.timezone('US/Eastern')

full['Timestamp'] = full['Timestamp'].dt.tz_convert(est_timezone)
full['Timestamp'] = full['Timestamp'].dt.strftime('%Y-%m-%d')

In [291]:
full['Team_x'] = full['Team_x'].map(mapping)
full['Team_y'] = full['Team_y'].map(mapping)

In [292]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2021-01-20,06340b4723931e9471b3df5aa8b169a2,FanDuel,HOU,2.88,PHO,1.44
1,2021-01-20,06340b4723931e9471b3df5aa8b169a2,FanDuel,PHO,1.44,HOU,2.88
2,2021-01-20,06340b4723931e9471b3df5aa8b169a2,DraftKings,HOU,2.88,PHO,1.43
3,2021-01-20,06340b4723931e9471b3df5aa8b169a2,DraftKings,PHO,1.43,HOU,2.88
4,2021-01-20,06340b4723931e9471b3df5aa8b169a2,BetMGM,HOU,2.90,PHO,1.40
...,...,...,...,...,...,...,...
23645,2024-03-20,def47031598222435edee8a26a792e3f,DraftKings,MIL,4.90,BOS,1.19
23646,2024-03-20,def47031598222435edee8a26a792e3f,BetMGM,BOS,1.18,MIL,5.00
23647,2024-03-20,def47031598222435edee8a26a792e3f,BetMGM,MIL,5.00,BOS,1.18
23648,2024-03-20,def47031598222435edee8a26a792e3f,FanDuel,BOS,1.17,MIL,5.30


In [293]:
fanduel = full[full['Sportsbook'] == 'FanDuel']
columns = fanduel.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Fanduel_odds_x','Team_y', 'Fanduel_odds_y']
fanduel.columns = columns
fanduel = fanduel.drop('Sportsbook', axis=1)

draftkings = full[full['Sportsbook'] == 'DraftKings']
columns = draftkings.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Draftkings_odds_x','Team_y', 'Draftkings_odds_y']
draftkings.columns = columns
draftkings = draftkings.drop('Sportsbook', axis=1)

betMGM = full[full['Sportsbook'] == 'BetMGM']
columns = betMGM.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'BetMGM_odds_x','Team_y', 'BetMGM_odds_y']
betMGM.columns = columns
betMGM = betMGM.drop('Sportsbook', axis=1)

In [294]:
merged_fd_dk = pd.merge(fanduel, draftkings, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

# Merge the result with betMGM DataFrame
final = pd.merge(merged_fd_dk, betMGM, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

In [295]:
columns = final.columns
columns = ['Timestamp', 'Id', 'Teams_x', 'Fanduel_odds_x','Teams_y', 'Fanduel_odds_y', 'Draftkings_odds_x', 'Draftkings_odds_y', 'BetMGM_odds_x', 'BetMGM_odds_y']
final.columns = columns
final = final.drop('Id', axis=1)

order = ['Timestamp', 'Teams_x', 'Fanduel_odds_x','Draftkings_odds_x', 'BetMGM_odds_x', 'Teams_y', 'Fanduel_odds_y','Draftkings_odds_y', 'BetMGM_odds_y']
final = final[order]

In [296]:
final

,Timestamp,Teams_x,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Teams_y,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y
0,2021-01-20,HOU,2.88,2.88,2.90,PHO,1.44,1.43,1.40
1,2021-01-20,PHO,1.44,1.43,1.40,HOU,2.88,2.88,2.90
2,2021-01-20,MEM,1.86,1.89,2.00,POR,1.98,1.93,1.83
3,2021-01-20,POR,1.98,1.93,1.83,MEM,1.86,1.89,2.00
4,2021-01-20,GSW,1.86,1.88,1.83,SAS,1.98,1.94,2.00
...,...,...,...,...,...,...,...,...,...
7691,2024-03-20,POR,6.60,6.25,6.50,LAC,1.12,1.13,1.12
7692,2024-03-20,CLE,1.77,1.77,1.77,MIA,2.10,2.10,2.05
7693,2024-03-20,MIA,2.10,2.10,2.05,CLE,1.77,1.77,1.77
7694,2024-03-20,BOS,1.17,1.19,1.18,MIL,5.30,4.90,5.00


In [297]:
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/'
file_name = '2021_01_20_onward.csv'
file_path = save_path + file_name
final.to_csv(file_path)